In [1]:
import numpy as np
import lightgbm as lgb  # Changed import
from sklearn.model_selection import StratifiedKFold
from helper_lin import get_all_data
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

# 1. Data Preparation
X_avg, y_avg = get_all_data()
selector = SelectKBest(score_func=f_classif, k=100)
X_reduced = selector.fit_transform(X_avg, y_avg)
scaler = StandardScaler()
X_final = scaler.fit_transform(X_reduced)

# 2. Setup the fold strategy
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

# 3. Cross-Validation Loop
for train_index, test_index in skf.split(X_final, y_avg):
    X_train, X_test = X_final[train_index], X_final[test_index]
    y_train, y_test = y_avg[train_index], y_avg[test_index]
    
    # Initialize LightGBM Classifier
    # Note: 'verbosity=-1' silences warnings/info messages in LightGBM
    model = lgb.LGBMClassifier(
        n_estimators=100, 
        random_state=42,
        importance_type='gain', # Optional: makes feature importance similar to XGB
        verbosity=-1 
    )
    
    model.fit(X_train, y_train)
    
    # Evaluate
    score = model.score(X_test, y_test)
    scores.append(score)

print(f"LightGBM Mean Accuracy: {np.mean(scores):.4f}")

LightGBM Mean Accuracy: 0.5000


/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fit

In [2]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from helper_lin import get_all_data
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

# 1. Data Preparation (Keep your existing pipeline)
X_avg, y_avg = get_all_data()
selector = SelectKBest(score_func=f_classif, k=100)
X_reduced = selector.fit_transform(X_avg, y_avg)
scaler = StandardScaler()
X_final = scaler.fit_transform(X_reduced)

# 2. Define the Parameter Grid
# These are the most impactful parameters to tune for LightGBM
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [15, 31, 63],
    'max_depth': [-1, 5, 10],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# 3. Initialize the Model
lgbm = lgb.LGBMClassifier(random_state=42, verbosity=-1)

# 4. Setup Grid Search
# This replaces your manual loop
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=lgbm, 
    param_grid=param_grid, 
    cv=cv, 
    scoring='accuracy', 
    n_jobs=-1
)

# 5. Execute Tuning
grid_search.fit(X_final, y_avg)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Mean Accuracy: {grid_search.best_score_:.4f}")

/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fit

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': -1, 'n_estimators': 50, 'num_leaves': 15, 'subsample': 0.8}
Best Mean Accuracy: 0.5000


/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/fmriprep/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fit